In [1]:
!which python

/Users/amanmeda/anaconda3/envs/mlpm/bin/python


In [2]:
import pandas as pd

In [3]:
import os
os.getcwd()

'/Users/amanmeda/Documents/ml-pm-starter/notebooks'

In [4]:
import yfinance as yf

In [5]:
yf.download("AAPL", start="2018-01-01", progress=False).head()

/var/folders/st/y2tgd9yj129138_bv71m8m040000gn/T/ipykernel_71385/1135310102.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  yf.download("AAPL", start="2018-01-01", progress=False).head()


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2018-01-02,40.380993,40.390372,39.677736,39.888715,102223600
2018-01-03,40.373959,40.917812,40.310668,40.444285,118071600
2018-01-04,40.561508,40.664653,40.338811,40.446642,89738400
2018-01-05,41.023300,41.110034,40.566184,40.657607,94640000
2018-01-08,40.870934,41.166300,40.772474,40.870934,82271200


In [6]:
with open("../data/universe/Russell3000_holdings.xlsx", "rb") as f:
    first_bytes = f.read(200)
print(first_bytes[:200])


b'\xef\xbb\xbf\xef\xbb\xbf<?xml version="1.0"?>\n<ss:Workbook xmlns:ss="urn:schemas-microsoft-com:office:spreadsheet">\n<ss:Styles>\n<ss:Style ss:ID="Default">\n<ss:Alignment ss:Horizontal="Left"/>\n</ss:Style>\n<ss:Style ss:I'


In [7]:
import re
import pandas as pd
from bs4 import BeautifulSoup

path = "../data/universe/Russell3000_holdings.xlsx"   # your SpreadsheetML file

# Read text (handles BOM)
with open(path, "r", encoding="utf-8-sig", errors="ignore") as f:
    xml_text = f.read()

soup = BeautifulSoup(xml_text, "xml")

def extract_rows_with_ss_index(soup):
    rows = soup.find_all("Row")
    table = []
    for r in rows:
        row_vals = []
        col = 1  # 1-based in SpreadsheetML
        for c in r.find_all("Cell", recursive=False):
            # Honor ss:Index (or Index) to pad missing cells
            idx_attr = c.get("ss:Index") or c.get("Index")
            if idx_attr:
                target = int(idx_attr)
                while col < target:
                    row_vals.append("")
                    col += 1
            # Cell value (display text)
            val = (c.Data.string or "").strip() if c.Data else ""
            row_vals.append(val)
            col += 1
        table.append(row_vals)
    return pd.DataFrame(table)

df_raw = extract_rows_with_ss_index(soup)

# Find the header row containing 'Ticker'
hdr_row = df_raw.index[df_raw.apply(lambda s: s.astype(str).str.fullmatch(r".*Ticker.*", case=False).any(), axis=1)]
if len(hdr_row) == 0:
    raise RuntimeError("Couldn't find header row with 'Ticker'.")
hdr_row = hdr_row[0]

header = df_raw.iloc[hdr_row].tolist()
df = df_raw.iloc[hdr_row+1:].reset_index(drop=True)
df.columns = header

# --- Clean & filter to get real US equity tickers ---
# Keep likely equities (if columns exist), otherwise skip gracefully
if "Asset Class" in df.columns:
    df = df[df["Asset Class"].str.contains("Equity", na=False)]
if "Currency" in df.columns:
    df = df[df["Currency"].eq("USD")]

# Ticker column name sometimes has weird spacing; find the first col that looks like 'Ticker'
ticker_col = next((c for c in df.columns if str(c).strip().lower() == "ticker"), None)
if ticker_col is None:
    raise RuntimeError("Couldn't find a 'Ticker' column after parsing.")

tickers = (
    df[ticker_col]
      .astype(str)
      .str.strip()
      # Remove obvious non-tickers like dates or long text
      .pipe(lambda s: s[~s.str.match(r"\d{4}-\d{2}-\d{2}")])
      # Typical US ticker patterns (allow BRK.B, RDS-A, etc.)
      .pipe(lambda s: s[s.str.fullmatch(r"[A-Z][A-Z0-9\.\-]{0,9}")] )
      .drop_duplicates()
      .sort_values()
)

print(f"Parsed {len(tickers)} tickers. Sample:", tickers.head(10).tolist())

out_csv = "../data/universe/russell3000_tickers.csv"
tickers.to_csv(out_csv, index=False)
print("Saved:", out_csv)


Parsed 2567 tickers. Sample: ['A', 'AA', 'AAL', 'AAMI', 'AAOI', 'AAON', 'AAP', 'AAPL', 'AAT', 'ABBV']
Saved: ../data/universe/russell3000_tickers.csv


In [8]:
#df = pd.read_csv('../data/sp500_tickers.csv')

In [9]:
df = pd.read_csv('../data/universe/russell3000_tickers.csv')
df.head()

,Ticker
0,A
1,AA
2,AAL
3,AAMI
4,AAOI


In [10]:
tickers = df['Ticker'].astype(str).apply(lambda x: x.strip()).tolist()

In [11]:
len(tickers)

2567

In [12]:
tickers = list(dict.fromkeys(tickers + ['SPY']))

In [13]:
len(tickers)

2568

In [14]:
import time
import pandas as pd
import yfinance as yf

batch_size   = 150
max_retries  = 3
data_list    = []
all_failed   = []

def to_close(df):
    """Return a DataFrame of Close prices (columns=tickers)."""
    if isinstance(df, pd.DataFrame) and isinstance(df.columns, pd.MultiIndex):
        # yfinance multiindex: ('Close', 'AAPL') or ('Adj Close', 'AAPL')
        if 'Close' in df.columns.get_level_values(0):
            return df['Close']
        elif 'Adj Close' in df.columns.get_level_values(0):
            return df['Adj Close']
        else:
            return pd.DataFrame()  # nothing usable
    elif isinstance(df, pd.DataFrame):
        # Sometimes yfinance returns a single-column DF already
        return df
    elif isinstance(df, pd.Series):
        # Single ticker -> Series
        return df.to_frame()
    else:
        return pd.DataFrame()

for i in range(0, len(tickers), batch_size):
    orig_batch = tickers[i:i+batch_size]
    remaining  = list(orig_batch)    # tickers we still need
    print(f"\n📦 Batch {i//batch_size+1}: {len(orig_batch)} tickers")

    for attempt in range(1, max_retries+1):
        if not remaining:
            break

        print(f"  Attempt {attempt}: requesting {len(remaining)} tickers…")
        try:
            df = yf.download(
                tickers=remaining,
                start="2015-01-01",
                end=None,
                auto_adjust=True,          # optional
                group_by="column",         # default behavior
                progress=False,
                threads=True,
            )
        except Exception as e:
            print(f"  ❌ Request error: {e}")
            time.sleep(2)
            continue

        close_df = to_close(df)

        # Identify good (non-empty) tickers returned this attempt
        good = []
        if not close_df.empty:
            # keep only columns that exist and aren’t entirely NaN
            for t in list(close_df.columns):
                if t in remaining and not close_df[t].isna().all():
                    good.append(t)

        # Append successes from THIS attempt
        if good:
            data_list.append(close_df[good])
            print(f"  ✅ Got {len(good)} new tickers this attempt.")
        else:
            print("  ⚠️ No usable columns returned this attempt.")

        # Compute which tickers still need retry
        got_set = set(good)
        remaining = [t for t in remaining if t not in got_set]

        if remaining:
            print(f"  ↩️  Will retry {len(remaining)} tickers…")
            time.sleep(2)  # polite pause to avoid rate limits
        else:
            break

    # After retries, whatever is still remaining is permanently failed
    if remaining:
        print(f"  🚫 Permanently failed: {len(remaining)}")
        all_failed.extend(remaining)

# Combine all successes
stock_data = pd.concat(data_list, axis=1) if data_list else pd.DataFrame()
# Drop any completely empty columns (extra safety)
stock_data = stock_data.dropna(axis=1, how="all")

print(f"\n✅ Final columns: {stock_data.shape[1]}")
print(f"🚫 Permanently failed: {len(all_failed)}")
# Optional: save the failures
# pd.Series(all_failed).to_csv("../data/universe/failed_tickers.csv", index=False)



📦 Batch 1: 150 tickers
  Attempt 1: requesting 150 tickers…



1 Failed download:
['ADRO']: YFTzMissingError('possibly delisted; no timezone found')


  ✅ Got 149 new tickers this attempt.
  ↩️  Will retry 1 tickers…
  Attempt 2: requesting 1 tickers…



1 Failed download:
['ADRO']: YFTzMissingError('possibly delisted; no timezone found')


  ⚠️ No usable columns returned this attempt.
  ↩️  Will retry 1 tickers…


...

  ✅ Got 148 new tickers this attempt.
  ↩️  Will retry 2 tickers…



2 Failed downloads:
['UHALB', 'THRD']: YFTzMissingError('possibly delisted; no timezone found')


  Attempt 2: requesting 2 tickers…
  ⚠️ No usable columns returned this attempt.
  ↩️  Will retry 2 tickers…



2 Failed downloads:
['UHALB', 'THRD']: YFTzMissingError('possibly delisted; no timezone found')


  Attempt 3: requesting 2 tickers…
  ⚠️ No usable columns returned this attempt.
  ↩️  Will retry 2 tickers…
  🚫 Permanently failed: 2

📦 Batch 17: 150 tickers
  Attempt 1: requesting 150 tickers…
  ✅ Got 150 new tickers this attempt.

📦 Batch 18: 18 tickers
  Attempt 1: requesting 18 tickers…
  ✅ Got 18 new tickers this attempt.

✅ Final columns: 2555
🚫 Permanently failed: 13


In [15]:
stock_data.tail()

Ticker,A,AA,AAL,AAMI,AAOI,AAON,AAP,AAPL,AAT,ABBV,...,ZG,ZION,ZIP,ZM,ZS,ZTS,ZUMZ,ZVRA,ZWS,ZYME
Date,,,,,,,,,,,,,,,,,,,,,
2025-10-27,146.589996,39.900002,13.41,49.529999,37.220001,104.970001,56.320000,268.809998,20.080000,228.029999,...,74.139999,52.849998,4.82,85.169998,326.329987,147.529999,22.50,11.08,46.619999,18.469999
2025-10-28,145.850006,39.310001,12.71,49.520000,36.869999,103.339996,57.000000,269.000000,20.110001,227.539993,...,72.830002,52.240002,4.77,84.889999,328.980011,145.410004,22.33,11.08,46.029999,19.299999
2025-10-29,143.199997,38.910000,12.97,49.259998,35.480000,103.269997,55.130001,269.700012,18.950001,225.139999,...,68.870003,51.509998,4.58,83.320000,320.959991,143.940002,22.25,10.12,47.980000,19.379999
2025-10-30,143.360001,36.759998,12.78,49.700001,35.070000,100.660004,50.700001,271.399994,19.100000,228.199997,...,68.750000,51.939999,4.50,84.339996,322.040009,144.100006,22.08,10.14,46.730000,19.120001
2025-10-31,146.360001,36.790001,13.13,48.099998,35.560001,98.389999,47.130001,270.369995,19.110001,218.039993,...,71.529999,52.110001,4.65,87.230003,331.140015,144.089996,21.65,10.18,47.110001,19.084999


In [16]:
stock_data = stock_data.sort_index()

In [17]:
nan_summary = stock_data.isna().mean().sort_values(ascending=False)
bad_tickers = nan_summary[nan_summary == 1.0].index.tolist()

print(f"Total tickers: {len(stock_data.columns)}")
print(f"Completely empty tickers: {len(bad_tickers)}")

Total tickers: 2555
Completely empty tickers: 0


In [18]:
stock_data

Ticker,A,AA,AAL,AAMI,AAOI,AAON,AAP,AAPL,AAT,ABBV,...,ZG,ZION,ZIP,ZM,ZS,ZTS,ZUMZ,ZVRA,ZWS,ZYME
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,37.120953,35.668667,51.079918,14.812617,10.790000,13.626691,139.455231,24.261047,27.042574,42.091431,...,32.329281,21.507469,NaN,NaN,NaN,39.889088,38.209999,NaN,12.788544,NaN
2015-01-05,36.425388,33.602219,51.051495,14.610462,10.650000,13.166876,137.617050,23.577579,27.350708,41.299301,...,30.427191,20.701614,NaN,NaN,NaN,39.649620,38.939999,NaN,12.280484,NaN
2015-01-06,35.857948,33.849293,50.255581,14.215336,10.250000,12.955611,137.520294,23.579792,27.565062,41.094872,...,30.227627,19.910944,NaN,NaN,NaN,39.262802,38.459999,NaN,12.074511,NaN
2015-01-07,36.333881,34.725300,50.227154,14.279658,9.850000,13.073670,140.475449,23.910439,28.087563,42.755802,...,31.718117,20.101009,NaN,NaN,NaN,40.073292,40.279999,NaN,12.092821,NaN
2015-01-08,37.422974,35.713585,50.843040,14.252090,9.960000,13.396782,141.706757,24.829126,28.255024,43.202961,...,32.410351,20.382299,NaN,NaN,NaN,40.690365,41.400002,NaN,12.207246,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-10-27,146.589996,39.900002,13.410000,49.529999,37.220001,104.970001,56.320000,268.809998,20.080000,228.029999,...,74.139999,52.849998,4.82,85.169998,326.329987,147.529999,22.500000,11.08,46.619999,18.469999
2025-10-28,145.850006,39.310001,12.710000,49.520000,36.869999,103.339996,57.000000,269.000000,20.110001,227.539993,...,72.830002,52.240002,4.77,84.889999,328.980011,145.410004,22.330000,11.08,46.029999,19.299999
2025-10-29,143.199997,38.910000,12.970000,49.259998,35.480000,103.269997,55.130001,269.700012,18.950001,225.139999,...,68.870003,51.509998,4.58,83.320000,320.959991,143.940002,22.250000,10.12,47.980000,19.379999


In [20]:
monthly_stock_data = stock_data.resample('M').last()
monthly_stock_data.tail()

/var/folders/st/y2tgd9yj129138_bv71m8m040000gn/T/ipykernel_71385/2034586746.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_stock_data = stock_data.resample('M').last()


Ticker,A,AA,AAL,AAMI,AAOI,AAON,AAP,AAPL,AAT,ABBV,...,ZG,ZION,ZIP,ZM,ZS,ZTS,ZUMZ,ZVRA,ZWS,ZYME
Date,,,,,,,,,,,,,,,,,,,,,
2025-06-30,117.526001,29.409386,11.22,35.232704,25.690001,73.659409,46.064991,204.937408,19.428093,182.713303,...,68.489998,51.506039,5.01,77.980003,313.940002,155.430267,13.260000,8.81,36.497280,12.550000
2025-07-31,114.579918,29.867817,11.49,41.781349,22.870001,83.397430,52.806335,207.334702,18.719828,187.667099,...,76.750000,53.172001,4.20,74.050003,285.559998,145.789993,13.710000,11.11,44.162006,12.560000
2025-08-31,125.408180,32.189999,13.37,50.959446,24.200001,82.848106,60.686989,232.139999,20.559347,208.894058,...,81.480003,58.009998,5.00,81.419998,277.049988,156.399994,17.190001,9.07,45.360001,14.810000
2025-09-30,128.350006,32.889999,11.24,48.160000,25.930000,93.440002,61.094952,254.630005,20.320000,229.882751,...,74.440002,56.580002,4.22,82.500000,299.660004,146.320007,19.610001,9.51,47.029999,17.080000
2025-10-31,146.360001,36.790001,13.13,48.099998,35.560001,98.389999,47.130001,270.369995,19.110001,218.039993,...,71.529999,52.110001,4.65,87.230003,331.140015,144.089996,21.650000,10.18,47.110001,19.084999


In [21]:
monthly_stock_data

Ticker,A,AA,AAL,AAMI,AAOI,AAON,AAP,AAPL,AAT,ABBV,...,ZG,ZION,ZIP,ZM,ZS,ZTS,ZUMZ,ZVRA,ZWS,ZYME
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-31,34.567513,35.152058,46.503471,13.930476,8.940000,13.552128,139.842163,25.998575,29.728743,38.841816,...,30.221392,18.215586,NaN,NaN,NaN,39.428909,37.290001,NaN,11.328436,NaN
2015-02-28,38.631035,33.280483,45.477367,16.503386,12.480000,13.987087,136.262527,28.618616,27.477982,38.938362,...,35.781105,20.360027,NaN,NaN,NaN,42.529335,38.820000,NaN,12.619189,NaN
2015-03-31,38.120663,29.072611,50.110546,17.128235,13.880000,15.242258,131.706619,27.720806,29.158113,37.676899,...,31.275335,20.557991,NaN,NaN,NaN,42.713890,40.250000,NaN,12.216402,NaN
2015-04-30,37.955528,30.197714,45.941624,17.789841,14.280000,14.894291,125.820335,27.881214,26.813610,41.960949,...,30.445900,21.578276,NaN,NaN,NaN,41.061920,31.709999,176.639999,12.124860,NaN
2015-05-31,37.790386,28.189255,40.309532,17.890926,17.860001,14.707878,134.812546,29.145319,26.510439,43.213413,...,28.497038,22.034763,NaN,NaN,NaN,46.007469,29.860001,197.119995,11.726647,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,117.526001,29.409386,11.220000,35.232704,25.690001,73.659409,46.064991,204.937408,19.428093,182.713303,...,68.489998,51.506039,5.01,77.980003,313.940002,155.430267,13.260000,8.810000,36.497280,12.550000
2025-07-31,114.579918,29.867817,11.490000,41.781349,22.870001,83.397430,52.806335,207.334702,18.719828,187.667099,...,76.750000,53.172001,4.20,74.050003,285.559998,145.789993,13.710000,11.110000,44.162006,12.560000
2025-08-31,125.408180,32.189999,13.370000,50.959446,24.200001,82.848106,60.686989,232.139999,20.559347,208.894058,...,81.480003,58.009998,5.00,81.419998,277.049988,156.399994,17.190001,9.070000,45.360001,14.810000


In [22]:
monthly_stock_data.to_parquet('../data/prices_monthly.parquet')

In [23]:
test = pd.read_parquet('../data/prices_monthly.parquet')
test.head()

Ticker,A,AA,AAL,AAMI,AAOI,AAON,AAP,AAPL,AAT,ABBV,...,ZG,ZION,ZIP,ZM,ZS,ZTS,ZUMZ,ZVRA,ZWS,ZYME
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-31,34.567513,35.152058,46.503471,13.930476,8.940000,13.552128,139.842163,25.998575,29.728743,38.841816,...,30.221392,18.215586,NaN,NaN,NaN,39.428909,37.290001,NaN,11.328436,NaN
2015-02-28,38.631035,33.280483,45.477367,16.503386,12.480000,13.987087,136.262527,28.618616,27.477982,38.938362,...,35.781105,20.360027,NaN,NaN,NaN,42.529335,38.820000,NaN,12.619189,NaN
2015-03-31,38.120663,29.072611,50.110546,17.128235,13.880000,15.242258,131.706619,27.720806,29.158113,37.676899,...,31.275335,20.557991,NaN,NaN,NaN,42.713890,40.250000,NaN,12.216402,NaN
2015-04-30,37.955528,30.197714,45.941624,17.789841,14.280000,14.894291,125.820335,27.881214,26.813610,41.960949,...,30.445900,21.578276,NaN,NaN,NaN,41.061920,31.709999,176.639999,12.124860,NaN
2015-05-31,37.790386,28.189255,40.309532,17.890926,17.860001,14.707878,134.812546,29.145319,26.510439,43.213413,...,28.497038,22.034763,NaN,NaN,NaN,46.007469,29.860001,197.119995,11.726647,NaN
